In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential,Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout,Input

X = []
y = []
classes = 43
cur_path = 'D:\BTL'

#Truy xuất hình ảnh và giá trị của chúng
for i in range(classes):
    path = os.path.join(cur_path,'train',str(i))
    images = os.listdir(path)

    for a in images:
        try:
            image = Image.open(path + '\\'+ a)
            image = image.resize((32,32))
            image = np.array(image)
            #sim = Image.fromarray(image)
            X.append(image)
            y.append(i)
        except:
            print("Error loading image")

#Chuyển đổi danh sách thành mảng numpy
X = np.array(X)
y = np.array(y)

print(X.shape, y.shape)

#Tách tập dữ liệu đào tạo và thử nghiệm
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(39209, 32, 32, 3) (39209,)
(31367, 32, 32, 3) (7842, 32, 32, 3) (31367,) (7842,)


In [2]:
#onehot-encoding
y_train = to_categorical(y_train, 43)
y_test = to_categorical(y_test, 43)

In [3]:
#Xây dựng model

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=X_train.shape[1:]))
model.add(Conv2D(filters=32, kernel_size=(5,5), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 32)        2432      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 64)          0

In [4]:
#Đào tạo model 
epochs = 10
history = model.fit(X_train, y_train, batch_size=64, epochs=epochs, validation_data=(X_test, y_test))

Epoch 1/10
491/491 [==============================] - 151s 306ms/step - loss: 4.3495 - accuracy: 0.2053 - val_loss: 0.9202 - val_accuracy: 0.7665
Epoch 2/10
491/491 [==============================] - 145s 295ms/step - loss: 1.0936 - accuracy: 0.6711 - val_loss: 0.3680 - val_accuracy: 0.9111
Epoch 3/10
491/491 [==============================] - 142s 290ms/step - loss: 0.7086 - accuracy: 0.7828 - val_loss: 0.3891 - val_accuracy: 0.8931
Epoch 4/10
491/491 [==============================] - 135s 275ms/step - loss: 0.5335 - accuracy: 0.8371 - val_loss: 0.2922 - val_accuracy: 0.9235
Epoch 5/10
491/491 [==============================] - 130s 265ms/step - loss: 0.4073 - accuracy: 0.8762 - val_loss: 0.1413 - val_accuracy: 0.9637
Epoch 6/10
491/491 [==============================] - 138s 281ms/step - loss: 0.3470 - accuracy: 0.8948 - val_loss: 0.1351 - val_accuracy: 0.9600
Epoch 7/10
491/491 [==============================] - 140s 284ms/step - loss: 0.3079 - accuracy: 0.9079 - val_loss: 0.0941 -

In [7]:
#kiểm tra độ chính xác trên bộ dữ liệu thử nghiệm
from sklearn.metrics import accuracy_score

y_test = pd.read_csv('D:/BTL/Test.csv')

y_true = y_test["ClassId"].values
imgs = y_test["Path"].values

data=[]

for img in imgs:
    image = Image.open(img)
    image = image.resize((32,32))
    data.append(np.array(image))

X_test=np.array(data)

y_pred = model.predict_classes(X_test)

#Accuracy with the test data
from sklearn.metrics import accuracy_score
print(accuracy_score(y_true, y_pred))

0.9413301662707838
